In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.impute import SimpleImputer
from geopy.distance import geodesic
import lightgbm as lgb
import warnings
import joblib  # Add this import
from sklearn.cluster import MiniBatchKMeans

warnings.filterwarnings('ignore')

In [11]:
np.random.seed(42)

# Preprocessing functions
def process_missing_values(df, imputer_report=None):
    if imputer_report is None:
        imputer_report = {
            'dropped': [],
            'filled_0': [],
            'filled_N': [],
            'geo_imputed': {},
            'text_imputed': [],
            'single_impute': {},
            'complete_columns': []
        }

    df = df.drop(columns=['IATA_CODE_x'])
    imputer_report['dropped'].append('IATA_CODE_x')

    num_cols = df.select_dtypes(include=np.number).columns
    imputer = SimpleImputer(strategy='constant', fill_value=0)
    imputer.fit(df[num_cols])

    delay_cols = ['AIR_SYSTEM_DELAY', 'WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY', 
                    'AIRLINE_DELAY', 'SECURITY_DELAY']
    for col in delay_cols:
        df[col] = df[col].fillna(0)
    imputer_report['filled_0'].extend(delay_cols)

    df['CANCELLATION_REASON'] = df['CANCELLATION_REASON'].fillna('N')
    imputer_report['filled_N'].append('CANCELLATION_REASON')

    geo_cols = {
        'DEST_LAT': df['DEST_LAT'].median(),
        'DEST_LON': df['DEST_LON'].mean(),
        'ORIGIN_LAT': df['ORIGIN_LAT'].median(),
        'ORIGIN_LON': df['ORIGIN_LON'].mean()
    }
    for col, val in geo_cols.items():
        df[col] = df[col].fillna(val)
    imputer_report['geo_imputed'] = geo_cols

    text_cols = ['IATA_CODE_y', 'IATA_CODE', 'DESTINATION_CITY', 'DESTINATION_STATE',
                'ORIGIN_CITY', 'ORIGIN_STATE', 'ORIGIN_AIRPORT_NORMALIZED',
                'DESTINATION_AIRPORT_NORMALIZED', 'TAIL_NUMBER']
    for col in text_cols:
        df[col] = df[col].fillna('UNKNOWN')
    imputer_report['text_imputed'] = text_cols

    scheduled_time_median = df['SCHEDULED_TIME'].median()
    df['SCHEDULED_TIME'] = df['SCHEDULED_TIME'].fillna(scheduled_time_median)
    imputer_report['single_impute']['SCHEDULED_TIME'] = scheduled_time_median

    complete_cols = ['MONTH', 'DESTINATION_AIRPORT_CODE', 'ORIGIN_AIRPORT_CODE',
                   'DAY', 'Flight_Status', 'DESTINATION_AIRPORT', 'ORIGIN_AIRPORT',
                   'FLIGHT_NUMBER', 'AIRLINE', 'DISTANCE']
    imputer_report['complete_columns'] = complete_cols

    return df, imputer_report, imputer

def create_airport_clusters(df, n_clusters=100):
    origins = df[['ORIGIN_AIRPORT_NORMALIZED', 'ORIGIN_LAT', 'ORIGIN_LON']].drop_duplicates()
    destinations = df[['DESTINATION_AIRPORT_NORMALIZED', 'DEST_LAT', 'DEST_LON']].drop_duplicates()
    destinations.columns = ['ORIGIN_AIRPORT_NORMALIZED', 'ORIGIN_LAT', 'ORIGIN_LON']
    all_airports = pd.concat([origins, destinations]).drop_duplicates('ORIGIN_AIRPORT_NORMALIZED')

    coords = all_airports[['ORIGIN_LAT', 'ORIGIN_LON']].values
    model = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
    model.fit(coords)
    joblib.dump(model, 'airport_cluster_model.pkl')
    return model

def calculate_airport_importance(df):
    origin_counts = df['ORIGIN_AIRPORT_NORMALIZED'].value_counts()
    dest_counts = df['DESTINATION_AIRPORT_NORMALIZED'].value_counts()
    total_counts = origin_counts.add(dest_counts, fill_value=0)
    return {
        'large': set(total_counts.nlargest(20).index),
        'medium': set(total_counts.nlargest(100).index) - set(total_counts.nlargest(20).index),
        'small': set(total_counts.index) - set(total_counts.nlargest(100).index)
    }

def add_airport_features(df, cluster_model, importance_map):
    df['ORIGIN_CLUSTER'] = cluster_model.predict(df[['ORIGIN_LAT', 'ORIGIN_LON']].values)
    df['DEST_CLUSTER'] = cluster_model.predict(df[['DEST_LAT', 'DEST_LON']].values)

    df['ORIGIN_IMPORTANCE'] = df['ORIGIN_AIRPORT_NORMALIZED'].apply(
        lambda x: 'large' if x in importance_map['large'] else 
                  'medium' if x in importance_map['medium'] else 'small')
    df['DEST_IMPORTANCE'] = df['DESTINATION_AIRPORT_NORMALIZED'].apply(
        lambda x: 'large' if x in importance_map['large'] else 
                  'medium' if x in importance_map['medium'] else 'small')

    df['DISTANCE_KM'] = df.apply(lambda row: geodesic(
        (row['ORIGIN_LAT'], row['ORIGIN_LON']),
        (row['DEST_LAT'], row['DEST_LON'])).km, axis=1)

    df['ORIGIN_STATE'] = df['ORIGIN_STATE'].str.upper()
    df['DESTINATION_STATE'] = df['DESTINATION_STATE'].str.upper()

    df['SAME_STATE'] = (df['ORIGIN_STATE'] == df['DESTINATION_STATE']).astype(int)

    return df

def calculate_airline_stats(df):
    stats = df.groupby('AIRLINE')['Flight_Status'].agg(['mean', 'count'])
    stats.columns = ['airline_cancellation_rate', 'airline_flight_count']
    return stats

def add_airline_features(df, airline_stats):
    df = df.merge(airline_stats, how='left', left_on='AIRLINE', right_index=True)
    df['airline_cancellation_rate'].fillna(airline_stats['airline_cancellation_rate'].mean(), inplace=True)
    df['airline_flight_count'].fillna(1, inplace=True)
    return df

def add_date_features(df):
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']].assign(YEAR=2020))
    df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek
    df['SEASON'] = df['MONTH'].apply(lambda x: (x % 12 + 3) // 3)
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5, 6]).astype(int)
    df.drop(columns=['DATE'], inplace=True)
    return df

def select_and_encode_features(df):
    features = [
        'MONTH', 'DAY', 'DAY_OF_WEEK', 'SEASON', 'IS_WEEKEND',
        'SCHEDULED_TIME', 'DISTANCE_KM',
        'AIRLINE', 'airline_cancellation_rate', 'airline_flight_count',
        'ORIGIN_CLUSTER', 'DEST_CLUSTER', 
        'ORIGIN_IMPORTANCE', 'DEST_IMPORTANCE',
        'SAME_STATE'
    ]
    df = df[features]

    categoricals = ['AIRLINE', 'ORIGIN_IMPORTANCE', 'DEST_IMPORTANCE', 'SEASON']
    df = pd.get_dummies(df, columns=categoricals, drop_first=True)
    return df



In [ ]:
# Pipeline function
def preprocess_pipeline(input_csv_path, output_file_path):
    df = pd.read_csv(input_csv_path)
    df, _, _ = process_missing_values(df)
    cluster_model = create_airport_clusters(df)
    importance_map = calculate_airport_importance(df)
    df = add_airport_features(df, cluster_model, importance_map)
    airline_stats = calculate_airline_stats(df)
    df = add_airline_features(df, airline_stats)
    if 'YEAR' not in df.columns:
        df['YEAR'] = 2020
    df = add_date_features(df)
    df_encoded = select_and_encode_features(df)

    df_encoded['Flight_Status'] = df['Flight_Status'].values  # Add label back
    df_encoded.to_csv(output_file_path, index=False)
    print(f"Preprocessing complete. Output saved to: {output_file_path}")


In [15]:
preprocess_pipeline(input_csv_path="../output/flights_cleaned.csv", output_file_path="../output/flights_preprocessed.csv")

Preprocessing complete. Output saved to: ../output/flights_preprocessed.csv


In [3]:
# Load data
df = pd.read_csv("../output/flights_cleaned.csv")

In [4]:
df.columns

Index(['MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_TIME',
       'Flight_Status', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY', 'DISTANCE', 'IATA_CODE_x', 'ORIGIN_AIRPORT_CODE',
       'DESTINATION_AIRPORT_CODE', 'ORIGIN_AIRPORT_NORMALIZED',
       'DESTINATION_AIRPORT_NORMALIZED', 'IATA_CODE_y', 'ORIGIN_CITY',
       'ORIGIN_STATE', 'ORIGIN_LAT', 'ORIGIN_LON', 'IATA_CODE',
       'DESTINATION_CITY', 'DESTINATION_STATE', 'DEST_LAT', 'DEST_LON'],
      dtype='object')

In [6]:
df.head(5)

,MONTH,DAY,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,Flight_Status,CANCELLATION_REASON,...,IATA_CODE_y,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_LAT,ORIGIN_LON,IATA_CODE,DESTINATION_CITY,DESTINATION_STATE,DEST_LAT,DEST_LON
0,1,1,MQ,3303,N678MQ,DFW,GJT,133.0,0,NaN,...,DFW,Dallas-Fort Worth,TX,32.89595,-97.03720,GJT,Grand Junction,CO,39.12241,-108.52673
1,1,1,NK,295,N521NK,ORD,IAH,167.0,0,NaN,...,ORD,Chicago,IL,41.97960,-87.90446,IAH,Houston,TX,29.98047,-95.33972
2,1,1,OO,2610,N866AS,LAX,FAT,55.0,0,NaN,...,LAX,Los Angeles,CA,33.94254,-118.40807,FAT,Fresno,CA,36.77619,-119.71814
3,1,1,OO,5343,N584SW,LAX,SBP,59.0,0,NaN,...,LAX,Los Angeles,CA,33.94254,-118.40807,SBP,San Luis Obispo,CA,35.23706,-120.64239
4,1,1,OO,4443,N455SW,CVG,MKE,83.0,0,NaN,...,CVG,Covington,KY,39.04614,-84.66217,MKE,Milwaukee,WI,42.94722,-87.89658


In [61]:
def analyze_missing_values(df, sample_size=5):
    """
    Analyzes missing values and recommends imputation strategies.
    Returns a DataFrame with analysis results and recommended strategies.
    
    Parameters:
    - df: Input DataFrame
    - sample_size: Number of sample values to show for each column
    """
    analysis = pd.DataFrame(index=df.columns)
    
    # Basic stats
    analysis['dtype'] = df.dtypes
    analysis['missing_count'] = df.isnull().sum()
    analysis['missing_pct'] = (analysis['missing_count'] / len(df)) * 100
    analysis['unique_count'] = df.nunique()
    
    # Value distribution analysis (numeric only)
    numeric_cols = df.select_dtypes(include=np.number).columns
    analysis['min'] = df[numeric_cols].min()
    analysis['median'] = df[numeric_cols].median()
    analysis['max'] = df[numeric_cols].max()
    
    # Mode (works for all types)
    analysis['mode'] = df.mode().iloc[0] if not df.mode().empty else np.nan
    
    # Sample values with NaN handling
    def get_samples(col):
        non_null = df[col].dropna()
        if len(non_null) == 0:
            return ["ALL_NULL"]
        sample_count = min(sample_size, len(non_null))
        return non_null.sample(sample_count, random_state=42).tolist()
    
    analysis['sample_values'] = [get_samples(col) for col in df.columns]
    
    # Recommend imputation strategy
    def recommend_strategy(col):
        if analysis.loc[col, 'missing_count'] == 0:
            return 'no missing values'
        
        if analysis.loc[col, 'missing_pct'] == 100:
            return 'drop_column (all null)'
        
        dtype = analysis.loc[col, 'dtype']
        
        # Categorical columns
        if dtype == 'object' or dtype.name == 'category':
            unique_count = analysis.loc[col, 'unique_count']
            if unique_count <= 10:
                return "most_frequent (categorical)"
            return "constant '' (free-text)"
        
        # Numeric columns
        else:
            if col not in numeric_cols:
                return "check_dtype (non-numeric)"
            
            min_val = analysis.loc[col, 'min']
            max_val = analysis.loc[col, 'max']
            
            # Binary flags (0/1)
            unique_vals = df[col].dropna().unique()
            if set(unique_vals).issubset({0, 1}):
                return "constant 0 (binary)"
            
            # Percentage-like (0-100)
            elif 0 <= min_val and max_val <= 100:
                return "median (percentage-like)"
            
            # Count data (non-negative)
            elif min_val >= 0:
                if max_val > 100:  # Large counts
                    return "median (large counts)"
                return "constant 0 (small counts)"
            
            # Continuous with negatives
            else:
                spread = max_val - min_val
                if spread > 10 * df[col].std():  # Potential outliers
                    return "median (robust to outliers)"
                return "mean (normal distribution)"
    
    analysis['recommended_strategy'] = [recommend_strategy(col) for col in df.columns]
    
    # Add example imputation code
    def example_code(col):
        strategy = analysis.loc[col, 'recommended_strategy']
        if strategy == 'drop_column (all null)':
            return "df.drop(columns=['" + col + "'])"
        elif 'most_frequent' in strategy:
            return f"df['{col}'].fillna(df['{col}'].mode()[0])"
        elif 'median' in strategy:
            return f"df['{col}'].fillna(df['{col}'].median())"
        elif 'mean' in strategy:
            return f"df['{col}'].fillna(df['{col}'].mean())"
        elif 'constant 0' in strategy:
            return f"df['{col}'].fillna(0)"
        elif "constant ''" in strategy:
            return f"df['{col}'].fillna('')"
        else:
            return "No imputation needed"
    
    analysis['example_code'] = [example_code(col) for col in df.columns]
    
    return analysis.sort_values('missing_pct', ascending=False)



In [63]:
# Usage example:
missing_analysis = analyze_missing_values(df)
missing_analysis

,dtype,missing_count,missing_pct,unique_count,min,median,max,mode,sample_values,recommended_strategy,example_code
IATA_CODE_x,float64,581746,100.000000,0,NaN,NaN,NaN,NaN,[ALL_NULL],drop_column (all null),df.drop(columns=['IATA_CODE_x'])
CANCELLATION_REASON,object,570758,98.111203,4,NaN,NaN,NaN,B,"[A, B, A, C, B]",most_frequent (categorical),df['CANCELLATION_REASON'].fillna(df['CANCELLAT...
AIR_SYSTEM_DELAY,float64,445682,76.611098,274,0.00000,0.00000,643.00000,0.0,"[0.0, 9.0, 18.0, 0.0, 0.0]",median (large counts),df['AIR_SYSTEM_DELAY'].fillna(df['AIR_SYSTEM_D...
WEATHER_DELAY,float64,445682,76.611098,329,0.00000,0.00000,819.00000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",median (large counts),df['WEATHER_DELAY'].fillna(df['WEATHER_DELAY']...
LATE_AIRCRAFT_DELAY,float64,445682,76.611098,372,0.00000,13.00000,1174.00000,0.0,"[40.0, 0.0, 12.0, 0.0, 35.0]",median (large counts),df['LATE_AIRCRAFT_DELAY'].fillna(df['LATE_AIRC...
AIRLINE_DELAY,float64,445682,76.611098,453,0.00000,3.00000,1323.00000,0.0,"[0.0, 9.0, 26.0, 17.0, 0.0]",median (large counts),df['AIRLINE_DELAY'].fillna(df['AIRLINE_DELAY']...
SECURITY_DELAY,float64,445682,76.611098,88,0.00000,0.00000,221.00000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",median (large counts),df['SECURITY_DELAY'].fillna(df['SECURITY_DELAY...
DEST_LON,float64,46402,7.976333,292,-170.71053,-90.25803,-64.97336,-118.40807,"[-75.24114, -85.736, -97.66987, -74.16866, -84...",mean (normal distribution),df['DEST_LON'].fillna(df['DEST_LON'].mean())
DEST_LAT,float64,46402,7.976333,292,14.33102,37.64996,71.28545,33.94254,"[39.87195, 38.17439, 30.19453, 40.6925, 39.04614]",median (percentage-like),df['DEST_LAT'].fillna(df['DEST_LAT'].median())
ORIGIN_LON,float64,45879,7.886432,201,-170.71053,-93.66068,-64.97336,-84.42694,"[-80.15275, -115.15233, -97.0372, -118.40807, ...",mean (normal distribution),df['ORIGIN_LON'].fillna(df['ORIGIN_LON'].mean())


In [50]:
df['Flight_Status'].value_counts()

0    570758
1     10988
Name: Flight_Status, dtype: int64

In [ ]:

# Preprocessing functions
def process_missing_values(df, imputer_report=None):
    """
    Enhanced version that works with your existing pipeline
    Returns:
    - df: Processed DataFrame
    - imputer_report: Dictionary of all imputations applied
    - imputer: Fitted SimpleImputer for numerical columns (for consistency with your original code)
    """
    
    # Initialize report if not provided
    if imputer_report is None:
        imputer_report = {
            'dropped': [],
            'filled_0': [],
            'filled_N': [],
            'geo_imputed': {},
            'text_imputed': [],
            'single_impute': {},
            'complete_columns': []
        }
    
    # 1. Drop completely null columns
    df = df.drop(columns=['IATA_CODE_x', 'IATA_CODE_y','AIR_SYSTEM_DELAY', 'WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY', 
                 'AIRLINE_DELAY', 'SECURITY_DELAY','CANCELLATION_REASON','IATA_CODE'])
    imputer_report['dropped'].append(['IATA_CODE_x', 'IATA_CODE_y','AIR_SYSTEM_DELAY', 'WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY', 
                 'AIRLINE_DELAY', 'SECURITY_DELAY','CANCELLATION_REASON','IATA_CODE'])
    
    # 2. Create numerical imputer for consistency with original code
    num_cols = df.select_dtypes(include=np.number).columns
    imputer = SimpleImputer(strategy='constant', fill_value=0)
    imputer.fit(df[num_cols])

    
    # 5. Geographic coordinates
    geo_cols = {
        'DEST_LAT': df['DEST_LAT'].median(),
        'DEST_LON': df['DEST_LON'].mean(),
        'ORIGIN_LAT': df['ORIGIN_LAT'].median(),
        'ORIGIN_LON': df['ORIGIN_LON'].mean()
    }
    for col, val in geo_cols.items():
        df[col] = df[col].fillna(val)
    imputer_report['geo_imputed'] = geo_cols
    
    # 6. Text fields
    text_cols = [ 'DESTINATION_CITY', 'DESTINATION_STATE',
               'ORIGIN_CITY', 'ORIGIN_STATE', 'ORIGIN_AIRPORT_NORMALIZED',
               'DESTINATION_AIRPORT_NORMALIZED', 'TAIL_NUMBER']
    for col in text_cols:
        df[col] = df[col].fillna('UNKNOWN')
    imputer_report['text_imputed'] = text_cols
    
    # 7. Scheduled time
    scheduled_time_median = df['SCHEDULED_TIME'].median()
    df['SCHEDULED_TIME'] = df['SCHEDULED_TIME'].fillna(scheduled_time_median)
    imputer_report['single_impute']['SCHEDULED_TIME'] = scheduled_time_median
    
    # 8. Complete columns tracking
    complete_cols = ['MONTH', 'DESTINATION_AIRPORT_CODE', 'ORIGIN_AIRPORT_CODE',
                   'DAY', 'Flight_Status', 'DESTINATION_AIRPORT', 'ORIGIN_AIRPORT',
                   'FLIGHT_NUMBER', 'AIRLINE', 'DISTANCE']
    imputer_report['complete_columns'] = complete_cols
    
    return df, imputer_report, imputer
def create_airport_clusters(df, n_clusters=100):
    # Use normalized airport names only
    origins = df[['ORIGIN_AIRPORT_NORMALIZED', 'ORIGIN_LAT', 'ORIGIN_LON']].drop_duplicates()
    destinations = df[['DESTINATION_AIRPORT_NORMALIZED', 'DEST_LAT', 'DEST_LON']].drop_duplicates()
    destinations.columns = ['ORIGIN_AIRPORT_NORMALIZED', 'ORIGIN_LAT', 'ORIGIN_LON']
    all_airports = pd.concat([origins, destinations]).drop_duplicates('ORIGIN_AIRPORT_NORMALIZED')
    
    coords = all_airports[['ORIGIN_LAT', 'ORIGIN_LON']].values
    model = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
    model.fit(coords)
    return model

def calculate_airport_importance(df):
    # Calculate based on normalized names only
    origin_counts = df['ORIGIN_AIRPORT_NORMALIZED'].value_counts()
    dest_counts = df['DESTINATION_AIRPORT_NORMALIZED'].value_counts()
    total_counts = origin_counts.add(dest_counts, fill_value=0)
    return {
        'large': set(total_counts.nlargest(20).index),
        'medium': set(total_counts.nlargest(100).index) - set(total_counts.nlargest(20).index),
        'small': set(total_counts.index) - set(total_counts.nlargest(100).index)
    }

def add_airport_features(df, cluster_model, importance_map):
    # Add cluster features
    df['ORIGIN_CLUSTER'] = cluster_model.predict(df[['ORIGIN_LAT', 'ORIGIN_LON']].values)
    df['DEST_CLUSTER'] = cluster_model.predict(df[['DEST_LAT', 'DEST_LON']].values)
    
    # Add importance features
    df['ORIGIN_IMPORTANCE'] = df['ORIGIN_AIRPORT_NORMALIZED'].apply(
        lambda x: 'large' if x in importance_map['large'] else 
'medium' if x in importance_map['medium'] else 'small')
    df['DEST_IMPORTANCE'] = df['DESTINATION_AIRPORT_NORMALIZED'].apply(
        lambda x: 'large' if x in importance_map['large'] else 
'medium' if x in importance_map['medium'] else 'small')

    # Add distance feature (using both provided distance and calculated geodesic distance)
    df['DISTANCE_KM'] = df.apply(lambda row: geodesic(
        (row['ORIGIN_LAT'], row['ORIGIN_LON']),
        (row['DEST_LAT'], row['DEST_LON'])).km, axis=1)
    
    # Add state features
    df['ORIGIN_STATE'] = df['ORIGIN_STATE'].str.upper()
    df['DESTINATION_STATE'] = df['DESTINATION_STATE'].str.upper()
    
    # Add same-state flight indicator
    df['SAME_STATE'] = (df['ORIGIN_STATE'] == df['DESTINATION_STATE']).astype(int)
    
    return df

def calculate_airline_stats(df):
    # Pre-compute airline cancellation rates (historical data)
    stats = df.groupby('AIRLINE')['Flight_Status'].agg(['mean', 'count'])
    stats.columns = ['airline_cancellation_rate', 'airline_flight_count']
    return stats

def add_airline_features(df, airline_stats):
    # Merge airline stats
    df = df.merge(airline_stats, how='left', left_on='AIRLINE', right_index=True)
    df['airline_cancellation_rate'].fillna(airline_stats['airline_cancellation_rate'].mean(), inplace=True)
    df['airline_flight_count'].fillna(1, inplace=True)
    return df

def add_date_features(df):
    # Add day of week and season
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']].assign(YEAR=2020))  # Using dummy year
    df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek
    df['SEASON'] = df['MONTH'].apply(lambda x: (x%12 + 3)//3)
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5, 6]).astype(int)
    
    return df

def select_and_encode_features(df):
    features = [
        'MONTH', 'DAY', 'DAY_OF_WEEK', 'SEASON', 'IS_WEEKEND',
        'SCHEDULED_TIME', 'DISTANCE', 'DISTANCE_KM',
        'AIRLINE', 'airline_cancellation_rate', 'airline_flight_count',
        'ORIGIN_CLUSTER', 'DEST_CLUSTER', 
        'ORIGIN_IMPORTANCE', 'DEST_IMPORTANCE',
        'SAME_STATE'
    ]
    df = df[features]

    # One-hot encode categoricals
    categoricals = ['AIRLINE', 'ORIGIN_IMPORTANCE', 'DEST_IMPORTANCE', 'SEASON']
    df = pd.get_dummies(df, columns=categoricals, drop_first=True)
    return df


In [66]:
if 'YEAR' not in df.columns:
        df['YEAR'] = 2020

    # Process missing values
df, imputer_report, imputer = process_missing_values(df)

# Feature engineering before split
importance_map = calculate_airport_importance(df)
cluster_model = create_airport_clusters(df)
airline_stats = calculate_airline_stats(df)

df = add_date_features(df)
df = add_airport_features(df, cluster_model, importance_map)
df = add_airline_features(df, airline_stats)



In [69]:
df.to_csv('../output/encoded_features.csv', index=False)

In [68]:
# Split data before encoding and label separation
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Flight_Status'], random_state=42)
# Drop labels and encode features

y_train = train_df['Flight_Status']
X_train = train_df.drop(columns=['Flight_Status'])

y_test = test_df['Flight_Status']
X_test = test_df.drop(columns=['Flight_Status'])

X_train = select_and_encode_features(X_train)
X_test = select_and_encode_features(X_test)


In [36]:
df.head(5)

,MONTH,DAY,DAY_OF_WEEK,IS_WEEKEND,SCHEDULED_TIME,DISTANCE,DISTANCE_KM,airline_cancellation_rate,airline_flight_count,ORIGIN_CLUSTER,...,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_IMPORTANCE_medium,ORIGIN_IMPORTANCE_small,DEST_IMPORTANCE_medium,DEST_IMPORTANCE_small,SEASON_2.0,SEASON_3.0,SEASON_4.0
0,1.0,1.0,2,0,133.0,773.0,1243.487686,0.067339,24295,94,...,0,0,0,0,0,0,1,0,0,0
1,1.0,1.0,2,0,167.0,925.0,1489.248234,0.022309,18513,74,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1.0,2,0,55.0,209.0,336.166963,0.029653,45932,50,...,0,0,0,0,0,1,0,0,0,0
3,1.0,1.0,2,0,59.0,156.0,250.265898,0.029653,45932,50,...,0,0,0,0,0,0,1,0,0,0
4,1.0,1.0,2,0,83.0,318.0,511.514623,0.029653,45932,96,...,0,0,0,1,0,1,0,0,0,0


In [37]:
df.columns

Index(['MONTH', 'DAY', 'DAY_OF_WEEK', 'IS_WEEKEND', 'SCHEDULED_TIME',
       'DISTANCE', 'DISTANCE_KM', 'airline_cancellation_rate',
       'airline_flight_count', 'ORIGIN_CLUSTER', 'DEST_CLUSTER', 'SAME_STATE',
       'Flight_Status', 'AIRLINE_AS', 'AIRLINE_B6', 'AIRLINE_DL', 'AIRLINE_EV',
       'AIRLINE_F9', 'AIRLINE_HA', 'AIRLINE_MQ', 'AIRLINE_NK', 'AIRLINE_OO',
       'AIRLINE_UA', 'AIRLINE_US', 'AIRLINE_VX', 'AIRLINE_WN',
       'ORIGIN_IMPORTANCE_medium', 'ORIGIN_IMPORTANCE_small',
       'DEST_IMPORTANCE_medium', 'DEST_IMPORTANCE_small', 'SEASON_2.0',
       'SEASON_3.0', 'SEASON_4.0'],
      dtype='object')

In [38]:
#df.to_csv('../output/encoded_features.csv', index=False)

In [12]:
df.describe(include='all')

,MONTH,DAY,DAY_OF_WEEK,IS_WEEKEND,SCHEDULED_TIME,DISTANCE,DISTANCE_KM,airline_cancellation_rate,airline_flight_count,ORIGIN_CLUSTER,...,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_IMPORTANCE_medium,ORIGIN_IMPORTANCE_small,DEST_IMPORTANCE_medium,DEST_IMPORTANCE_small,SEASON_2.0,SEASON_3.0,SEASON_4.0
count,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,...,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000,535288.000000
mean,6.226624,15.765511,2.927361,0.264547,143.731720,877.730425,1404.537055,0.019900,62898.174097,51.608913,...,0.042450,0.010374,0.223487,0.260719,0.014149,0.434346,0.147468,0.277309,0.286457,0.179173
std,3.385623,8.815874,1.994413,0.441092,84.090177,721.215326,1161.306244,0.014783,36255.565322,25.147974,...,0.201614,0.101323,0.416582,0.439028,0.118107,0.495671,0.354572,0.447671,0.452106,0.383497
min,1.000000,1.000000,0.000000,0.000000,0.000000,49.000000,0.000000,0.001071,5553.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,8.000000,1.000000,0.000000,80.000000,349.000000,555.938638,0.013855,34026.000000,33.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,16.000000,3.000000,0.000000,115.000000,621.000000,984.790226,0.017863,46048.000000,50.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,23.000000,5.000000,1.000000,178.000000,1107.000000,1781.217777,0.020992,80187.000000,74.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,12.000000,31.000000,6.000000,1.000000,587.000000,4962.000000,7986.352258,0.067339,119630.000000,99.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535288 entries, 0 to 581745
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   MONTH                      535288 non-null  float64
 1   DAY                        535288 non-null  float64
 2   DAY_OF_WEEK                535288 non-null  int64  
 3   IS_WEEKEND                 535288 non-null  int32  
 4   SCHEDULED_TIME             535288 non-null  float64
 5   DISTANCE                   535288 non-null  float64
 6   DISTANCE_KM                535288 non-null  float64
 7   airline_cancellation_rate  535288 non-null  float64
 8   airline_flight_count       535288 non-null  int64  
 9   ORIGIN_CLUSTER             535288 non-null  int32  
 10  DEST_CLUSTER               535288 non-null  int32  
 11  SAME_STATE                 535288 non-null  int32  
 12  Flight_Status              535288 non-null  float64
 13  AIRLINE_AS                 53

In [22]:
if 'YEAR' not in df.columns:
    df['YEAR'] = 2020  # Using dummy year
df, imputer = process_missing_values(df)

# Step 2: Preprocessing that needs the full dataset
importance_map = calculate_airport_importance(df)
cluster_model = create_airport_clusters(df)
airline_stats = calculate_airline_stats(df)

# Step 3: Add features BEFORE splitting
df = add_date_features(df)
df = add_airport_features(df, cluster_model, importance_map)
df = add_airline_features(df, airline_stats)



In [30]:
df['Flight_Status']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
581741    0.0
581742    0.0
581743    0.0
581744    0.0
581745    0.0
Name: Flight_Status, Length: 535288, dtype: float64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

def data_quality_report(df):
    """Generate a comprehensive data quality report"""
    print("="*80)
    print("DATA QUALITY REPORT")
    print("="*80)
    
    # Basic info
    print("\nBASIC INFORMATION:")
    print(f"Total rows: {len(df)}")
    print(f"Total columns: {len(df.columns)}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum()/1024/1024:.2f} MB")
    
    # Missing values analysis
    print("\nMISSING VALUES ANALYSIS:")
    missing = df.isnull().sum()
    missing_percent = missing/len(df)*100
    missing_report = pd.concat([missing, missing_percent], axis=1)
    missing_report.columns = ['Missing Count', 'Missing %']
    print(missing_report[missing_report['Missing Count'] > 0].sort_values('Missing %', ascending=False))
    
    # Duplicates analysis
    print("\nDUPLICATES ANALYSIS:")
    print(f"Total duplicate rows: {df.duplicated().sum()}")
    
    # Target variable analysis
    if 'Flight_Status' in df.columns:
        print("\nTARGET VARIABLE ANALYSIS:")
        target_dist = df['Flight_Status'].value_counts(normalize=True)
        print(target_dist)
        plt.figure(figsize=(8,4))
        sns.countplot(x='Flight_Status', data=df)
        plt.title('Target Variable Distribution')
        plt.show()
    
    # Numeric features analysis
    numeric_cols = df.select_dtypes(include=np.number).columns
    if len(numeric_cols) > 0:
        print("\nNUMERIC FEATURES ANALYSIS:")
        print(df[numeric_cols].describe().T)
        
        # Plot distributions
        plt.figure(figsize=(15, 10))
        for i, col in enumerate(numeric_cols):
            plt.subplot(4, 4, i+1)
            sns.histplot(df[col], kde=True)
            plt.title(col)
        plt.tight_layout()
        plt.show()
    
    # Categorical features analysis
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(cat_cols) > 0:
        print("\nCATEGORICAL FEATURES ANALYSIS:")
        for col in cat_cols:
            print(f"\nColumn: {col}")
            print(f"Unique values: {df[col].nunique()}")
            if df[col].nunique() < 20:
                print(df[col].value_counts(normalize=True))
            else:
                print(f"Top 5 values:\n{df[col].value_counts(normalize=True).head()}")
    
    # Correlation analysis
    if len(numeric_cols) > 1:
        print("\nCORRELATION ANALYSIS:")
        plt.figure(figsize=(12, 8))
        sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
        plt.title('Feature Correlation Matrix')
        plt.show()

def check_feature_engineering(df):
    """Check if feature engineering was properly applied"""
    print("\n" + "="*80)
    print("FEATURE ENGINEERING CHECKS")
    print("="*80)
    
    required_features = [
        'ORIGIN_CLUSTER', 'DEST_CLUSTER', 'ORIGIN_IMPORTANCE', 
        'DEST_IMPORTANCE', 'DISTANCE_KM', 'airline_cancellation_rate',
        'DAY_OF_WEEK', 'SEASON', 'IS_WEEKEND'
    ]
    
    missing_features = [f for f in required_features if f not in df.columns]
    if missing_features:
        print("⚠️ Missing engineered features:", missing_features)
    else:
        print("✅ All expected engineered features present")
        
        # Check if features have proper distributions
        print("\nEngineered Feature Distributions:")
        for feat in required_features:
            if feat in df.columns:
                print(f"\n{feat}:")
                print(df[feat].value_counts(normalize=True).head())
                
                if feat in ['ORIGIN_CLUSTER', 'DEST_CLUSTER']:
                    if df[feat].nunique() != 100:
                        print(f"⚠️ Unexpected number of clusters: {df[feat].nunique()} (expected 100)")
                
                if feat in ['ORIGIN_IMPORTANCE', 'DEST_IMPORTANCE']:
                    expected_cats = ['large', 'medium', 'small']
                    if not all(cat in df[feat].unique() for cat in expected_cats):
                        print(f"⚠️ Missing importance categories. Expected: {expected_cats}")

def check_train_test_split(X_train, X_test, y_train, y_test):
    """Verify the train-test split was performed correctly"""
    print("\n" + "="*80)
    print("TRAIN-TEST SPLIT VALIDATION")
    print("="*80)
    
    # Check target distribution
    print("\nTarget Distribution:")
    print("Train:", Counter(y_train))
    print("Test:", Counter(y_test))
    
    # Check if split was stratified
    train_ratio = sum(y_train)/len(y_train)
    test_ratio = sum(y_test)/len(y_test)
    print(f"\nClass ratios - Train: {train_ratio:.4f}, Test: {test_ratio:.4f}")
    if abs(train_ratio - test_ratio) > 0.01:
        print("⚠️ Significant difference in class ratios - split may not be stratified")
    else:
        print("✅ Class ratios consistent - stratified split confirmed")
    
    # Check feature distributions
    print("\nFeature Distribution Comparison (sample feature):")
    sample_feature = X_train.columns[0]
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    sns.histplot(X_train[sample_feature], kde=True)
    plt.title(f'Train: {sample_feature}')
    plt.subplot(1, 2, 2)
    sns.histplot(X_test[sample_feature], kde=True)
    plt.title(f'Test: {sample_feature}')
    plt.tight_layout()
    plt.show()

def check_data_leakage(train_df, test_df):
    """Check for potential data leakage issues"""
    print("\n" + "="*80)
    print("DATA LEAKAGE CHECKS")
    print("="*80)
    
    # Check for overlapping indices
    train_indices = set(train_df.index)
    test_indices = set(test_df.index)
    overlap = train_indices.intersection(test_indices)
    print(f"Overlapping indices: {len(overlap)}")
    
    # Check temporal leakage
    if 'DATE' in train_df.columns:
        max_train_date = train_df['DATE'].max()
        min_test_date = test_df['DATE'].min()
        if max_train_date > min_test_date:
            print("⚠️ Temporal leakage detected - test data contains earlier dates than train data")
    
    # Check for target-related leakage
    if 'Flight_Status' in train_df.columns:
        # Check if features contain perfect predictors
        for col in train_df.columns:
            if col != 'Flight_Status':
                if (train_df[col].nunique() == 1) and (train_df[col].iloc[0] == train_df['Flight_Status'].iloc[0]):
                    print(f"⚠️ Potential leakage - {col} perfectly predicts target")

def full_data_diagnostic(df, X_train=None, X_test=None, y_train=None, y_test=None):
    """Run complete data diagnostic pipeline"""
    # Initial data quality check
    data_quality_report(df)
    
    # Feature engineering checks
    check_feature_engineering(df)
    
    # Train-test split validation
    if all(v is not None for v in [X_train, X_test, y_train, y_test]):
        check_train_test_split(X_train, X_test, y_train, y_test)
        
        # Data leakage checks
        train_df = pd.concat([X_train, y_train], axis=1)
        test_df = pd.concat([X_test, y_test], axis=1)
        check_data_leakage(train_df, test_df)
    
    print("\nDiagnostic complete. Review the outputs for any warnings or issues.")

